libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime


load client dataset

In [2]:
df = pd.read_csv("clean_data_after_eda.csv")

# Convert date columns
df["date_activ"] = pd.to_datetime(df["date_activ"])
df["date_end"] = pd.to_datetime(df["date_end"])
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"])
df["date_renewal"] = pd.to_datetime(df["date_renewal"])

df.head()


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0


Load price dataset (Estelle requirement)

In [5]:
price_df = pd.read_csv("price_data (1).csv")

price_df["price_date"] = pd.to_datetime(price_df["price_date"])

price_df.head()


,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


## FEATURE ENGINEERING


EStelle feature

In [6]:
# Sort by date
price_df = price_df.sort_values(["id", "price_date"])

# Get January prices
jan_prices = price_df.groupby("id").first().reset_index()

# Get December prices
dec_prices = price_df.groupby("id").last().reset_index()

# Calculate price difference
price_diff = pd.merge(
    jan_prices[["id", "price_off_peak_var"]],
    dec_prices[["id", "price_off_peak_var"]],
    on="id",
    suffixes=("_jan", "_dec")
)

price_diff["offpeak_price_change_dec_jan"] = (
    price_diff["price_off_peak_var_dec"]
    - price_diff["price_off_peak_var_jan"]
)

# Keep only required columns
price_diff = price_diff[["id", "offpeak_price_change_dec_jan"]]

price_diff.head()


,id,offpeak_price_change_dec_jan
0,0002203ffbb812588b632b9e628cc38d,-0.006192
1,0004351ebdd665e6ee664792efc4fd13,-0.004104
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018
4,00114d74e963e47177db89bc70108537,-0.003994


mergeing with main dataset

In [7]:
df = pd.merge(df, price_diff, on="id", how="left")

df.head()


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,offpeak_price_change_dec_jan
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,0.020057
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,-0.003767
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,-0.004670
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,-0.004547
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,-0.006192


tenure feature 

In [8]:
df["tenure"] = (
    (df["date_end"] - df["date_activ"]).dt.days // 365
)


convert dates to months


In [9]:
reference_date = datetime(2016, 1, 1)

def convert_months(reference_date, df, column):

    dates = pd.to_datetime(df[column])

    year_diff = reference_date.year - dates.dt.year
    month_diff = reference_date.month - dates.dt.month

    return year_diff * 12 + month_diff


df["months_activ"] = convert_months(reference_date, df, "date_activ")

df["months_to_end"] = convert_months(reference_date, df, "date_end")

df["months_modif_prod"] = convert_months(reference_date, df, "date_modif_prod")

df["months_renewal"] = convert_months(reference_date, df, "date_renewal")


convert categorical to numeric

In [10]:
df["has_gas"] = df["has_gas"].replace(['t','f'], [1,0])

df = pd.get_dummies(df, columns=["channel_sales", "origin_up"])


C:\Users\user\AppData\Local\Temp\ipykernel_19512\80337110.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["has_gas"] = df["has_gas"].replace(['t','f'], [1,0])


drop orginal date cols 

In [11]:
df = df.drop(columns=[
    "date_activ",
    "date_end",
    "date_modif_prod",
    "date_renewal"
])


final check

In [12]:
df.head()

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 58 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              14606 non-null  object 
 1   cons_12m                                        14606 non-null  int64  
 2   cons_gas_12m                                    14606 non-null  int64  
 3   cons_last_month                                 14606 non-null  int64  
 4   forecast_cons_12m                               14606 non-null  float64
 5   forecast_cons_year                              14606 non-null  int64  
 6   forecast_discount_energy                        14606 non-null  float64
 7   forecast_meter_rent_12m                         14606 non-null  float64
 8   forecast_price_energy_off_peak                  14606 non-null  float64
 9   forecast_price_energy_peak             

Feature Engineering Summary

• Created tenure feature from date_activ and date_end to represent how long the customer stayed with the company.

• Created months_activ, months_to_end, months_modif_prod, and months_renewal features to represent customer lifecycle and contract timeline.

• Created offpeak_price_change_dec_jan feature using price_data.csv to represent change in off-peak electricity price between December and January.

• Converted has_gas feature from categorical format (t, f) into numeric format (1, 0) so it can be used in modeling.

• Converted categorical features such as channel_sales and origin_up into numeric format using one-hot encoding.

• Removed original date columns after extracting useful information from them.

• Final dataset now contains meaningful numerical features representing customer tenure, price behavior, and customer characteristics.

• These engineered features help improve the model’s ability to identify patterns related to customer churn.